# Imports & Global Config

In [1]:
import os
import gc
import sys
import time
import psutil
import shutil
import pickle
import random
import warnings
import torch
from datetime import datetime
from typing import Tuple, Dict, Any, List

import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix, save_npz, load_npz
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from IPython.display import display, Markdown

try:
    from tqdm import tqdm
except Exception:
    tqdm = lambda x, **k: x

try:
    from implicit.als import AlternatingLeastSquares
    implicit_available = True
except Exception:
    implicit_available = False

from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings("ignore")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

_NOTEBOOK_CREATED = datetime.now().isoformat()

# Utilities: logging, environment checks, cleanup

In [2]:
def log(msg: str, level: str = "INFO") -> None:
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{level}] {ts} | {msg}")

def compute_cache_strategy(max_fraction: float = 0.5, cap_gb: float = 16.0, safety_gb: float = 2.0) -> Tuple[str, int]:
    ram = psutil.virtual_memory()
    ram_total = ram.total / (1024**3)
    ram_free = ram.available / (1024**3)
    disk_free = shutil.disk_usage("/").free / (1024**3)
    budget = min(ram_total * max_fraction, cap_gb)
    if ram_free > (budget + safety_gb):
        mode = "ram"
    elif disk_free < 10:
        mode = "ram" 
    else:
        mode = "disk"
    workers = max(1, min(8, (psutil.cpu_count(logical=False) or 1) // (2 if mode == "disk" else 1)))
    return mode, workers

def set_vram_fraction(fraction: float = 0.5, device: int = 0) -> None:
    try:
        import torch
    except Exception:
        log("PyTorch not installed. Skipping VRAM fraction set.", "WARN")
        return
    if torch.cuda.is_available():
        try:
            torch.cuda.set_per_process_memory_fraction(fraction, device=device)
            log(f"GPU{device} per-process memory fraction set to {fraction:.2f}", "VRAM")
        except Exception as e:
            log(f"Cannot set VRAM fraction: {e}", "WARN")
    else:
        log("CUDA not available. CPU mode.", "INFO")

def enable_perf_flags() -> None:
    try:
        import torch
        if torch.cuda.is_available():
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            torch.backends.cudnn.benchmark = True
            log("Enabled TF32 & cudnn.benchmark", "PERF")
        else:
            log("CUDA not available (perf flags skipped)", "INFO")
    except Exception:
        log("PyTorch not installed; deprioritizing GPU optimizations", "WARN")

def clear_torch_caches(verbose: bool = False) -> None:
    gc.collect()
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            if verbose:
                log("Cleared CUDA cache", "CLEAN")
    except Exception:
        pass

CACHE_MODE, N_WORKERS = compute_cache_strategy()
log(f"Cache mode: {CACHE_MODE} | Workers: {N_WORKERS}")

enable_perf_flags()

[INFO] 2025-11-10 20:47:19 | Cache mode: ram | Workers: 6
[PERF] 2025-11-10 20:47:19 | Enabled TF32 & cudnn.benchmark


## Paths & Configuration

In [3]:
BASE_PATH = "processed"
PROCESSED_PATH = BASE_PATH
CLEANED_PATH = os.path.join(PROCESSED_PATH, "cleaned")
DATASETS_PATH = os.path.join(PROCESSED_PATH, "datasets")
PREPROCESS_PATH = os.path.join(PROCESSED_PATH, "preprocess")
EDA_PATH = os.path.join(PROCESSED_PATH, "eda")
MODEL_PATH = os.path.join(PROCESSED_PATH, "models")

os.makedirs(MODEL_PATH, exist_ok=True)

log("Directory check:")
for name, p in [
    ("BASE", BASE_PATH),
    ("PROCESSED", PROCESSED_PATH),
    ("CLEANED", CLEANED_PATH),
    ("DATASETS", DATASETS_PATH),
    ("PREPROCESS", PREPROCESS_PATH),
    ("EDA", EDA_PATH),
    ("MODEL", MODEL_PATH)
]:
    log(f"{name}: {'FOUND' if os.path.exists(p) else 'MISSING'}")

[INFO] 2025-11-10 20:47:19 | Directory check:
[INFO] 2025-11-10 20:47:19 | BASE: FOUND
[INFO] 2025-11-10 20:47:19 | PROCESSED: FOUND
[INFO] 2025-11-10 20:47:19 | CLEANED: FOUND
[INFO] 2025-11-10 20:47:19 | DATASETS: FOUND
[INFO] 2025-11-10 20:47:19 | PREPROCESS: FOUND
[INFO] 2025-11-10 20:47:19 | EDA: FOUND
[INFO] 2025-11-10 20:47:19 | MODEL: FOUND


## Utility: Load CSVs safely (with logging)

In [4]:
def load_csv(path: str, name: str = None) -> pd.DataFrame:
    if os.path.exists(path):
        df = pd.read_csv(path, low_memory=False)
        log(f"Loaded: {name or os.path.basename(path)} | shape={df.shape}")
        return df
    else:
        log(f"Missing file: {path}", "WARN")
        return None

FILES = {
    "movies": os.path.join(CLEANED_PATH, "movies_cleaned_f.csv"),
    "ratings": os.path.join(CLEANED_PATH, "ratings_cleaned_f.csv"),
    "tfidf_reduced": os.path.join(PREPROCESS_PATH, "movies_tfidf_reduced.csv"),
    "hybrid": os.path.join(PREPROCESS_PATH, "hybrid_user_movie_info.csv"),
    "user_map": os.path.join(PREPROCESS_PATH, "user_mapping.csv"),
    "movie_map": os.path.join(PREPROCESS_PATH, "movie_mapping.csv"),
    "user_movie_matrix_npz": os.path.join(MODEL_PATH, "user_movie_matrix.npz"),
}

movies = load_csv(FILES["movies"], "movies_cleaned_f.csv")
ratings = load_csv(FILES["ratings"], "ratings_cleaned_f.csv")
tfidf_reduced = load_csv(FILES["tfidf_reduced"], "movies_tfidf_reduced.csv")
hybrid = load_csv(FILES["hybrid"], "hybrid_user_movie_info.csv")
user_map = load_csv(FILES["user_map"], "user_mapping.csv")
movie_map = load_csv(FILES["movie_map"], "movie_mapping.csv")

required = {"movies": movies, "ratings": ratings}
for k, v in required.items():
    if v is None:
        raise FileNotFoundError(f"Required dataset '{k}' not found. ปรับ PATH แล้วรันใหม่")

log(f"Primary datasets shapes -> movies: {movies.shape}, ratings: {ratings.shape}")

[INFO] 2025-11-10 20:47:23 | Loaded: movies_cleaned_f.csv | shape=(87585, 23)
[INFO] 2025-11-10 20:47:53 | Loaded: ratings_cleaned_f.csv | shape=(32000204, 5)
[INFO] 2025-11-10 20:47:54 | Loaded: movies_tfidf_reduced.csv | shape=(87585, 51)
[INFO] 2025-11-10 20:48:08 | Loaded: hybrid_user_movie_info.csv | shape=(32000204, 7)
[INFO] 2025-11-10 20:48:08 | Loaded: user_mapping.csv | shape=(200948, 2)
[INFO] 2025-11-10 20:48:08 | Loaded: movie_mapping.csv | shape=(84432, 2)
[INFO] 2025-11-10 20:48:08 | Primary datasets shapes -> movies: (87585, 23), ratings: (32000204, 5)


## Dataset Quick Summary

In [5]:
def dataset_summary(dfs: Dict[str, pd.DataFrame]) -> None:
    for name, df in dfs.items():
        if df is None:
            log(f"{name}: MISSING", "WARN")
            continue
        log(f"--- {name.upper()} ---")
        log(f"Shape: {df.shape}")
        log(f"Columns: {df.columns.tolist()}")
        miss = int(df.isnull().sum().sum())
        log(f"Total missing values: {miss}")

dataset_summary({"movies": movies, "ratings": ratings, "tfidf_reduced": tfidf_reduced, "hybrid": hybrid})

[INFO] 2025-11-10 20:48:08 | --- MOVIES ---
[INFO] 2025-11-10 20:48:08 | Shape: (87585, 23)
[INFO] 2025-11-10 20:48:08 | Columns: ['movieId', 'title', 'year', '(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
[INFO] 2025-11-10 20:48:08 | Total missing values: 1
[INFO] 2025-11-10 20:48:08 | --- RATINGS ---
[INFO] 2025-11-10 20:48:08 | Shape: (32000204, 5)
[INFO] 2025-11-10 20:48:08 | Columns: ['userId', 'movieId', 'rating', 'timestamp', 'datetime']
[INFO] 2025-11-10 20:48:09 | Total missing values: 0
[INFO] 2025-11-10 20:48:09 | --- TFIDF_REDUCED ---
[INFO] 2025-11-10 20:48:09 | Shape: (87585, 51)
[INFO] 2025-11-10 20:48:09 | Columns: ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12', 'featur

## Prepare Sparse User-Movie Matrix (Memory efficient)

In [6]:
def build_user_movie_sparse(ratings_df: pd.DataFrame, save_path: str = None) -> Tuple[sparse.csr_matrix, Dict[int,int], Dict[int,int]]:
    log("Building user/movie index mappings...")
    unique_users = ratings_df['userId'].unique()
    unique_movies = ratings_df['movieId'].unique()
    user_index = {uid: i for i, uid in enumerate(unique_users)}
    movie_index = {mid: i for i, mid in enumerate(unique_movies)}
    rows = ratings_df['userId'].map(user_index).to_numpy()
    cols = ratings_df['movieId'].map(movie_index).to_numpy()
    data = ratings_df['rating'].to_numpy(dtype=np.float32)
    mat = sparse.coo_matrix((data, (rows, cols)), shape=(len(user_index), len(movie_index))).tocsr()
    if save_path:
        save_npz(save_path, mat)
        log(f"Saved sparse matrix to {save_path}")
    return mat, user_index, movie_index

sparse_matrix_path = FILES["user_movie_matrix_npz"]
if os.path.exists(sparse_matrix_path):
    log(f"Loading existing sparse matrix: {sparse_matrix_path}")
    user_movie_matrix = load_npz(sparse_matrix_path)
    user_index = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
    movie_index = {mid: i for i, mid in enumerate(ratings['movieId'].unique())}
else:
    user_movie_matrix, user_index, movie_index = build_user_movie_sparse(ratings, save_path=sparse_matrix_path)

log(f"User-movie matrix shape: {user_movie_matrix.shape} | nnz: {user_movie_matrix.nnz}")

[INFO] 2025-11-10 20:48:09 | Loading existing sparse matrix: processed\models\user_movie_matrix.npz
[INFO] 2025-11-10 20:48:10 | User-movie matrix shape: (200948, 84432) | nnz: 32000204


## Train/Test Split for CF & SVD

In [7]:
train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=SEED)
log(f"Train size: {train_df.shape}, Test size: {test_df.shape}")

[INFO] 2025-11-10 20:48:14 | Train size: (25600163, 5), Test size: (6400041, 5)


## Train SVD (scipy.sparse.linalg.svds)

In [9]:
def train_svd_model(train_df: pd.DataFrame, k: int = 50):
    log("Preparing sparse matrix for SVD training...")
    user_idx = {u: i for i, u in enumerate(train_df['userId'].unique())}
    movie_idx = {m: i for i, m in enumerate(train_df['movieId'].unique())}
    reverse_user_idx = {i: u for u, i in user_idx.items()}
    reverse_movie_idx = {i: m for m, i in movie_idx.items()}
    rows = train_df['userId'].map(user_idx).to_numpy()
    cols = train_df['movieId'].map(movie_idx).to_numpy()
    data = train_df['rating'].astype(np.float32).to_numpy()
    R = csr_matrix((data, (rows, cols)), shape=(len(user_idx), len(movie_idx)))
    log(f"Sparse R shape: {R.shape} | nnz={R.nnz}")
    user_sum = np.array(R.sum(axis=1)).flatten()
    user_counts = (R != 0).sum(axis=1).A1
    user_mean = np.nan_to_num(user_sum / np.where(user_counts == 0, 1, user_counts))
    log("Computing SVD (svds)... this may take a while depending on k and data size")
    U, sigma_vals, Vt = svds(R, k=k)
    sigma_vals_sorted_idx = np.argsort(sigma_vals)[::-1]
    sigma_vals = sigma_vals[sigma_vals_sorted_idx]
    U = U[:, sigma_vals_sorted_idx]
    Vt = Vt[sigma_vals_sorted_idx, :]
    Sigma = np.diag(sigma_vals)
    log(f"SVD done: U={U.shape} Sigma={Sigma.shape} Vt={Vt.shape}")
    return {
        "U": U, "Sigma": Sigma, "Vt": Vt,
        "user_index": user_idx, "movie_index": movie_idx,
        "reverse_user_index": reverse_user_idx, "reverse_movie_index": reverse_movie_idx,
        "user_mean": user_mean, "R": R
    }

svd_k = 50  
svd_artifacts = train_svd_model(train_df, k=svd_k)

U = svd_artifacts["U"]
Sigma = svd_artifacts["Sigma"]
Vt = svd_artifacts["Vt"]
svd_user_index = svd_artifacts["user_index"]
svd_movie_index = svd_artifacts["movie_index"]
svd_reverse_user_index = svd_artifacts["reverse_user_index"]
svd_reverse_movie_index = svd_artifacts["reverse_movie_index"]
svd_user_mean = svd_artifacts["user_mean"]
R_train = svd_artifacts["R"]

[INFO] 2025-11-10 20:55:47 | Preparing sparse matrix for SVD training...
[INFO] 2025-11-10 20:55:50 | Sparse R shape: (200948, 80318) | nnz=25600163
[INFO] 2025-11-10 20:55:50 | Computing SVD (svds)... this may take a while depending on k and data size
[INFO] 2025-11-10 20:55:56 | SVD done: U=(200948, 50) Sigma=(50, 50) Vt=(50, 80318)


## Predict function for SVD (on-demand)

In [10]:
def predict_for_user(user_id: int, top_n: int = 10):
    if user_id not in svd_user_index:
        raise ValueError("User ID not found in SVD training set.")
    u_idx = svd_user_index[user_id]
    user_vector = np.dot(U[u_idx, :], Sigma)  
    preds = np.dot(user_vector, Vt) + svd_user_mean[u_idx]  
    top_idx = np.argsort(preds)[::-1][:top_n]
    top_movie_ids = [svd_reverse_movie_index[i] for i in top_idx]
    top_scores = preds[top_idx]
    return pd.DataFrame({"movieId": top_movie_ids, "pred_rating": top_scores})

example_user = next(iter(svd_user_index.keys()))
log(f"Example SVD predict for user {example_user}")
display(predict_for_user(example_user, top_n=10).head())

[INFO] 2025-11-10 20:56:03 | Example SVD predict for user 161935


,movieId,pred_rating
0,318,9.245179
1,527,8.969478
2,79132,7.546449
3,1704,7.275461
4,2028,7.183497


## Evaluate SVD: RMSE (memory-efficient)

In [11]:
def rmse_sparse(true_df: pd.DataFrame, predict_func) -> float:
    true_vals = []
    pred_vals = []
    grouped = true_df.groupby('userId')
    for uid, group in grouped:
        try:
            preds_df = predict_func(uid, top_n=len(group))
            pred_dict = dict(zip(preds_df['movieId'], preds_df['pred_rating']))
        except ValueError:
            continue
        for row in group.itertuples(index=False):
            mid, r = row.movieId, row.rating
            p = pred_dict.get(mid, np.nan)
            if not np.isnan(p):
                true_vals.append(r)
                pred_vals.append(p)
    if not true_vals:
        raise ValueError("No predictions found for test set users (maybe different user space).")
    return sqrt(mean_squared_error(true_vals, pred_vals))

try:
    svd_rmse = rmse_sparse(test_df, predict_for_user)
    log(f"SVD RMSE = {svd_rmse:.4f}")
except Exception as e:
    log(f"RMSE calculation skipped or failed: {e}", "WARN")

[INFO] 2025-11-10 21:06:17 | SVD RMSE = 2.4007


## Content-based: Build similarity sparse matrix (จาก TF-IDF reduced)

In [12]:
CONTENT_SIM_PATH = os.path.join(MODEL_PATH, "content_similarity_sparse.npz")
if tfidf_reduced is None:
    log("TF-IDF reduced data not found. Skipping content-based similarity build.", "WARN")
    sim_sparse = None
    movie_ids = np.array([])
else:
    if "movieId" not in tfidf_reduced.columns:
        raise ValueError("tfidf_reduced ต้องมีคอลัมน์ 'movieId'")
    features = tfidf_reduced.drop(columns=["movieId"]).astype(np.float32).to_numpy()
    movie_ids = tfidf_reduced["movieId"].to_numpy()
    n_movies = features.shape[0]
    top_k = 50  
    log(f"Building content similarity sparse matrix for {n_movies} movies...")
    sim_sparse = lil_matrix((n_movies, n_movies), dtype=np.float32)

    for i in tqdm(range(n_movies), desc="ContentSim"):
        vec = features[i:i+1]
        sims = cosine_similarity(vec, features).flatten()
        sims[i] = -np.inf
        top_idx = np.argpartition(-sims, top_k)[:top_k]
        top_idx_sorted = top_idx[np.argsort(-sims[top_idx])]
        sim_sparse.rows[i] = top_idx_sorted.tolist()
        sim_sparse.data[i] = sims[top_idx_sorted].tolist()
        if i % 1000 == 0 and i > 0:
            log(f"Processed {i}/{n_movies}")
    save_npz(CONTENT_SIM_PATH, sim_sparse.tocsr())
    log(f"Saved content similarity matrix to {CONTENT_SIM_PATH} | shape={sim_sparse.shape}")

[INFO] 2025-11-10 21:06:17 | Building content similarity sparse matrix for 87585 movies...


ContentSim:   1%|▋                                                               | 1016/87585 [00:09<13:20, 108.16it/s]

[INFO] 2025-11-10 21:06:26 | Processed 1000/87585


ContentSim:   2%|█▍                                                              | 2013/87585 [00:18<12:55, 110.41it/s]

[INFO] 2025-11-10 21:06:36 | Processed 2000/87585


...

[INFO] 2025-11-10 21:20:11 | Processed 86000/87585


ContentSim:  99%|██████████████████████████████████████████████████████████████▌| 87015/87585 [14:04<00:05, 100.99it/s]

[INFO] 2025-11-10 21:20:22 | Processed 87000/87585


ContentSim: 100%|███████████████████████████████████████████████████████████████| 87585/87585 [14:10<00:00, 102.99it/s]


[INFO] 2025-11-10 21:20:29 | Saved content similarity matrix to processed\models\content_similarity_sparse.npz | shape=(87585, 87585)


## Hybrid Scoring Function

In [13]:
def hybrid_score(userId: int, movieId: int, alpha: float = 0.7, top_k: int = 50) -> float:
    try:
        svd_pred_df = predict_for_user(userId, top_n=1000) 
        svd_row = svd_pred_df[svd_pred_df.movieId == movieId]
        svd_score = float(svd_row.pred_rating.values[0]) if not svd_row.empty else np.nan
    except Exception:
        svd_score = np.nan

    if sim_sparse is None or len(movie_ids) == 0:
        content_score = np.nan
    else:
        idx_arr = np.where(movie_ids == movieId)[0]
        if idx_arr.size == 0:
            content_score = np.nan
        else:
            idx = int(idx_arr[0])
            row = sim_sparse.rows[idx] if isinstance(sim_sparse, lil_matrix) else sim_sparse[idx].indices
            data = sim_sparse.data[idx] if isinstance(sim_sparse, lil_matrix) else sim_sparse[idx].data
            if len(data) == 0:
                content_score = np.nan
            else:
                content_score = float(np.nanmean(data[:top_k]))

    if np.isnan(svd_score) and np.isnan(content_score):
        return np.nan
    if np.isnan(svd_score):
        return content_score
    if np.isnan(content_score):
        return svd_score
    return alpha * svd_score + (1.0 - alpha) * content_score

try:
    test_mid = movies['movieId'].iloc[0]
    test_uid = train_df['userId'].iloc[0]
    s = hybrid_score(test_uid, test_mid)
    log(f"Hybrid score sample (user {test_uid}, movie {test_mid}) = {s:.4f}")
except Exception as e:
    log(f"Hybrid score test skipped: {e}", "WARN")

[INFO] 2025-11-10 21:20:29 | Hybrid score sample (user 161935, movie 1) = 3.6434


## Recommend Function (Hybrid)

In [14]:
def recommend_movies(userId: int, top_n: int = 10, alpha: float = 0.7, top_k_content: int = 50) -> pd.DataFrame:
    seen = set(ratings.loc[ratings.userId == userId, 'movieId'].unique())
    candidate_movie_ids = movie_ids.tolist() if (movie_ids is not None and len(movie_ids) > 0) else movies['movieId'].unique().tolist()
    candidates = [mid for mid in candidate_movie_ids if mid not in seen]
    scores = []
    for mid in tqdm(candidates, desc=f"Scoring user {userId}", disable=False):
        score = hybrid_score(userId, mid, alpha=alpha, top_k=top_k_content)
        if not np.isnan(score):
            scores.append((mid, score))
    if len(scores) == 0:
        log(f"No candidate scores for user {userId}", "WARN")
        return pd.DataFrame(columns=['movieId', 'title', 'hybrid_score'])
    scores.sort(key=lambda x: x[1], reverse=True)
    top_scores = scores[:top_n]
    top_movie_ids = [mid for mid, s in top_scores]
    result = movies[movies.movieId.isin(top_movie_ids)][['movieId', 'title']].copy()
    order = {mid: i for i, mid in enumerate(top_movie_ids)}
    result['hybrid_score'] = result['movieId'].map(lambda m: dict(top_scores).get(m))
    result['rank'] = result['movieId'].map(lambda m: order.get(m))
    result = result.sort_values('rank').drop(columns=['rank']).reset_index(drop=True)
    return result

sample_uid = int(ratings['userId'].sample(1, random_state=SEED).iloc[0])
log(f"Generating sample recommendations for user {sample_uid}")
display(recommend_movies(sample_uid, top_n=10))

[INFO] 2025-11-10 21:20:30 | Generating sample recommendations for user 66954


Scoring user 66954: 100%|███████████████████████████████████████████████████████| 87492/87492 [04:16<00:00, 340.70it/s]


,movieId,title,hybrid_score
0,1073,Willy Wonka & the Chocolate Factory,4.270014
1,95,Broken Arrow,4.186669
2,802,Phenomenon,4.047356
3,6,Heat,3.989511
4,805,"Time to Kill, A",3.963768
5,1356,Star Trek: First Contact,3.899463
6,1393,Jerry Maguire,3.878329
7,104,Happy Gilmore,3.797816
8,628,Primal Fear,3.773855
9,260,Star Wars: Episode IV - A New Hope,3.698215


## Save SVD Artifacts (for deployment / loading ภายหลัง)

In [15]:
def save_svd_artifacts(model_dir: str = MODEL_PATH):
    os.makedirs(model_dir, exist_ok=True)
    np.save(os.path.join(model_dir, "svd_U.npy"), U.astype(np.float32))
    np.save(os.path.join(model_dir, "svd_Sigma.npy"), Sigma.astype(np.float32))
    np.save(os.path.join(model_dir, "svd_Vt.npy"), Vt.astype(np.float32))
    np.save(os.path.join(model_dir, "svd_user_mean.npy"), svd_user_mean.astype(np.float32))
    with open(os.path.join(model_dir, "svd_user_index.pkl"), "wb") as f:
        pickle.dump(svd_user_index, f)
    with open(os.path.join(model_dir, "svd_movie_index.pkl"), "wb") as f:
        pickle.dump(svd_movie_index, f)
    with open(os.path.join(model_dir, "svd_reverse_user_index.pkl"), "wb") as f:
        pickle.dump(svd_reverse_user_index, f)
    with open(os.path.join(model_dir, "svd_reverse_movie_index.pkl"), "wb") as f:
        pickle.dump(svd_reverse_movie_index, f)
    log("Saved SVD artifacts to disk")

save_svd_artifacts()

[INFO] 2025-11-10 21:24:47 | Saved SVD artifacts to disk


## ALS (Implicit) — ถ้ามี library `implicit`

In [16]:
if implicit_available:
    try:
        matrix_path = sparse_matrix_path
        if not os.path.exists(matrix_path):
            log(f"User-movie matrix not found at {matrix_path}", "WARN")
        else:
            mat = load_npz(matrix_path)  # user x movie
            log(f"Loaded matrix shape {mat.shape} nnz={mat.nnz}")
            mat_csr = mat.tocsr()
            als_model = AlternatingLeastSquares(
                factors=50,
                regularization=0.1,
                iterations=20,
                use_gpu=True, 
                calculate_training_loss=True
            )
            als_model.fit(mat_csr.T * 1.0)
            with open(os.path.join(MODEL_PATH, "als_model.pkl"), "wb") as f:
                pickle.dump(als_model, f)
            log("ALS training complete and saved")
    except Exception as e:
        log(f"ALS training failed: {e}", "ERROR")
else:
    log("Library 'implicit' not installed. Skip ALS training.", "WARN")

[INFO] 2025-11-10 21:24:48 | Loaded matrix shape (200948, 84432) nnz=32000204
[ERROR] 2025-11-10 21:24:48 | ALS training failed: No CUDA extension has been built, can't train on GPU.


## Feature-Based Hybrid Model (RandomForest example)

In [17]:
if hybrid is None:
    log("Hybrid feature dataset not found. Skipping feature-based model.", "WARN")
else:
    candidate_features = ['year_norm', 'year_rated', 'month_rated', 'day_rated']
    features = [f for f in candidate_features if f in hybrid.columns]
    if not features:
        log("No matching features in hybrid dataset; define features or update candidate_features", "WARN")
    else:
        log(f"Training feature-based hybrid model with features: {features}")
        train_h, test_h = train_test_split(hybrid, test_size=0.2, random_state=SEED)
        rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=SEED)
        rf.fit(train_h[features], train_h['rating'])
        preds = rf.predict(test_h[features])
        rmse_val = sqrt(mean_squared_error(test_h['rating'], preds))
        log(f"Feature-based hybrid RMSE: {rmse_val:.4f}")
        with open(os.path.join(MODEL_PATH, "hybrid_feature_model.pkl"), "wb") as f:
            pickle.dump({"model": rf, "features": features}, f)
        log("Saved hybrid feature-based model")

[INFO] 2025-11-10 21:24:48 | Training feature-based hybrid model with features: ['year_norm', 'year_rated', 'month_rated', 'day_rated']
[INFO] 2025-11-10 21:43:40 | Feature-based hybrid RMSE: 1.0476
[INFO] 2025-11-10 21:43:50 | Saved hybrid feature-based model


## Export Check: ตรวจสอบว่า artifacts ที่ต้องใช้ในการ deploy ถูกสร้างครบ

In [18]:
expected_files = [
    "svd_U.npy",
    "svd_Sigma.npy",
    "svd_Vt.npy",
    "svd_user_mean.npy",
    "svd_user_index.pkl",
    "svd_movie_index.pkl",
    "content_similarity_sparse.npz",  
    "hybrid_feature_model.pkl",      
    "user_movie_matrix.npz"
]

log("Exported artifacts check:")
all_ok = True
for fname in expected_files:
    fpath = os.path.join(MODEL_PATH, fname)
    exists = os.path.exists(fpath)
    log(f"{fname:40} | {'FOUND' if exists else 'MISSING'}")
    if not exists:
        all_ok = False

if all_ok:
    log("All artifacts present and ready for deployment 🎉")
else:
    log("Some artifacts are missing; check missing items above", "WARN")

[INFO] 2025-11-10 21:43:50 | Exported artifacts check:
[INFO] 2025-11-10 21:43:50 | svd_U.npy                                | FOUND
[INFO] 2025-11-10 21:43:50 | svd_Sigma.npy                            | FOUND
[INFO] 2025-11-10 21:43:50 | svd_Vt.npy                               | FOUND
[INFO] 2025-11-10 21:43:50 | svd_user_mean.npy                        | FOUND
[INFO] 2025-11-10 21:43:50 | svd_user_index.pkl                       | FOUND
[INFO] 2025-11-10 21:43:50 | svd_movie_index.pkl                      | FOUND
[INFO] 2025-11-10 21:43:50 | content_similarity_sparse.npz            | FOUND
[INFO] 2025-11-10 21:43:50 | hybrid_feature_model.pkl                 | FOUND
[INFO] 2025-11-10 21:43:50 | user_movie_matrix.npz                    | FOUND
[INFO] 2025-11-10 21:43:50 | All artifacts present and ready for deployment 🎉


## Quick Utilities: Load saved SVD artifacts (ตัวอย่างการโหลดต่อจากไฟล์)

In [19]:
def load_svd_artifacts(model_dir: str = MODEL_PATH) -> Dict[str, Any]:
    log("Loading SVD artifacts from disk...")
    U = np.load(os.path.join(model_dir, "svd_U.npy"))
    Sigma = np.load(os.path.join(model_dir, "svd_Sigma.npy"))
    Vt = np.load(os.path.join(model_dir, "svd_Vt.npy"))
    user_mean = np.load(os.path.join(model_dir, "svd_user_mean.npy"))
    with open(os.path.join(model_dir, "svd_user_index.pkl"), "rb") as f:
        user_index = pickle.load(f)
    with open(os.path.join(model_dir, "svd_movie_index.pkl"), "rb") as f:
        movie_index = pickle.load(f)
    with open(os.path.join(model_dir, "svd_reverse_user_index.pkl"), "rb") as f:
        reverse_user_index = pickle.load(f)
    with open(os.path.join(model_dir, "svd_reverse_movie_index.pkl"), "rb") as f:
        reverse_movie_index = pickle.load(f)
    log("Loaded SVD artifacts")
    return {
        "U": U, "Sigma": Sigma, "Vt": Vt, "user_mean": user_mean,
        "user_index": user_index, "movie_index": movie_index,
        "reverse_user_index": reverse_user_index, "reverse_movie_index": reverse_movie_index
    }

In [20]:
artifacts = load_svd_artifacts(MODEL_PATH)
U, Sigma, Vt = artifacts["U"], artifacts["Sigma"], artifacts["Vt"]
user_index = artifacts["user_index"]
movie_index = artifacts["movie_index"]
rev_user_index = artifacts["reverse_user_index"]
rev_movie_index = artifacts["reverse_movie_index"]
user_mean = artifacts["user_mean"]

recommendations = recommend_movies(userId=42, top_n=10)
display(recommendations)

[INFO] 2025-11-10 22:12:29 | Loading SVD artifacts from disk...
[INFO] 2025-11-10 22:12:30 | Loaded SVD artifacts


Scoring user 42: 100%|██████████████████████████████████████████████████████████| 87537/87537 [04:21<00:00, 334.14it/s]


,movieId,title,hybrid_score
0,541,Blade Runner,4.378204
1,750,Dr. Strangelove or: How I Learned to Stop Worr...,4.275218
2,1214,Alien,4.273540
3,1136,Monty Python and the Holy Grail,4.239331
4,1097,E.T. the Extra-Terrestrial,4.126972
5,1197,"Princess Bride, The",4.093783
6,1200,Aliens,4.080624
7,1225,Amadeus,3.994549
8,1206,"Clockwork Orange, A",3.925565
9,1387,Jaws,3.922256
